# Лабораторная работа №7
### Выполнила Рындина Валерия, M33351

In [1]:
import re
import numpy as np
from nltk import ngrams
from collections import defaultdict

### Педобработка файла

In [2]:
def preprocessing(file_name):
    with open('data/' + file_name + '.txt', 'r') as f:
        with open('data/' + file_name + '_clear.txt', 'w') as res:
            str = f.read()

            result = ''.join(c.lower() if bool(re.search('[а-яА-Я.?! \n]', c)) else ' ' for c in str)
            result = re.sub(r'\s+', r' ', result)

            result = re.sub(r' *\n *', r'', result)
            result = re.sub(r'\n\.\n', r'', result)
            result = re.sub(r'\.+', r'.', result)

            res.write(result)

### Чтение данных, one-hot и ngrams преобразования

In [3]:
def index_to_one_hot(index, size):
    one_hot_list = np.zeros(shape=size)
    one_hot_list[index] = 1
    return one_hot_list

def one_hot(text, size, char_to_index, index_to_char):
    one_hot_list = []

    for char in text:
        if char not in char_to_index:
            index = len(char_to_index)
            char_to_index[char] = index
            index_to_char[index] = char
        one_hot_list.append(index_to_one_hot(char_to_index[char], size))

    return np.array(one_hot_list)

def data_to_X_Y(data, n, size):
    ngrams_text = list(ngrams(data, n + 1))
    size_ngrams = len(ngrams_text)

    x = np.empty(shape=[size_ngrams, n, size])
    y = np.empty(shape=[size_ngrams, size])
    for window, index in zip(ngrams_text, range(size_ngrams)):
        x[index] = np.array(window[:n])
        y[index] = np.array(window[-1])

    return np.array(x), np.array(y)

def read_data(file_name, n):
    preprocessing(file_name)
    with open('data/' + file_name + '_clear.txt', 'r') as f:
        text = f.read()
        size = len(set(text))
        char_to_index = defaultdict(lambda: list)
        index_to_char = defaultdict(lambda: ' ')

        one_hot_line = one_hot(text, size, char_to_index, index_to_char)
        x_line, y_line = data_to_X_Y(one_hot_line, n, size)

        return x_line, y_line, char_to_index, index_to_char

### LSTM моделька

In [4]:
import keras
from keras import Input
from keras.layers import LSTM, Dense

def get_model(x, y):
    model = keras.Sequential()
    model.add(Input(shape=x.shape[1:]))
    model.add(LSTM(256, return_sequences=True))
    model.add(LSTM(256))
    model.add(Dense(y.shape[1], activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(x, y, batch_size=128, epochs=10)
    return model

2022-05-24 21:44:26.223370: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-24 21:44:26.223392: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
file_name = 'Esenin_Sobranie_stihotvorenii'
n = 5
x, y, char_to_index, index_to_char = read_data(file_name, n)

In [6]:
model = get_model(np.array(x), np.array(y))

2022-05-24 21:44:28.031349: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-24 21:44:28.031371: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-24 21:44:28.031385: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (valrun): /proc/driver/nvidia/version does not exist
2022-05-24 21:44:28.031529: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10


2022-05-24 21:44:28.489176: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 78001920 exceeds 10% of free system memory.


847/847 [==============================] - 31s 34ms/step - loss: 2.5957 - accuracy: 0.2546
Epoch 2/10
847/847 [==============================] - 28s 33ms/step - loss: 2.2247 - accuracy: 0.3269
Epoch 3/10
847/847 [==============================] - 28s 33ms/step - loss: 2.0570 - accuracy: 0.3706
Epoch 4/10
847/847 [==============================] - 28s 33ms/step - loss: 1.9263 - accuracy: 0.4042
Epoch 5/10
847/847 [==============================] - 28s 33ms/step - loss: 1.8246 - accuracy: 0.4331
Epoch 6/10
847/847 [==============================] - 27s 32ms/step - loss: 1.7398 - accuracy: 0.4555
Epoch 7/10
847/847 [==============================] - 28s 33ms/step - loss: 1.6647 - accuracy: 0.4766
Epoch 8/10
847/847 [==============================] - 27s 32ms/step - loss: 1.5962 - accuracy: 0.4954
Epoch 9/10
847/847 [==============================] - 28s 33ms/step - loss: 1.5318 - accuracy: 0.5130
Epoch 10/10
847/847 [==============================] - 28s 33ms/step - loss: 1.4697 - accurac

### Предсказания

In [58]:
def random_generation(file_name, predict, n):
    with open('data/' + file_name + '_clear.txt', 'r') as f:
        text = f.read()
        sentences = text.split('.')
        sentences_count = len(sentences)

        size = len(char_to_index)
        index = np.random.randint(0, sentences_count - 1)

        print(sentences[index])
        text = sentences[index][:(n + 1)]

        tmp_x, tmp_y = data_to_X_Y(one_hot(text, size, char_to_index, index_to_char), n, size)
        text = text[:n]
        test = tmp_x
        print(text)

        for i in range(100):
            prediction = np.argmax(predict(test))
            next_char = index_to_char[prediction]
            text += next_char

            test = test[0][1:]
            test = np.concatenate([test, [index_to_one_hot(prediction, size)]])
            test = np.array([test])

        print(text)

In [59]:
random_generation(file_name, lambda x: model.predict(x), n)

 сердце неласково к шуму мыши скребут в уголке
 серд
1/1 [==============================] - 0s 13ms/step
 сердце не смотреть такой же под окнами в прозрачный под окнами в прозрачный под окнами в прозрачный под 


In [133]:
random_generation(file_name, lambda x: model.predict(x), n)

 помирись лишь в сердце со врагом и тебя блаженством ошафранит
 поми
1/1 [==============================] - 0s 13ms/step
 поминать. я с севера что я с севера что я с севера что я с севера что я с севера что я с севера что я с 


In [131]:
random_generation(file_name, lambda x: model.predict(x), n)

 скука
 скук
1/1 [==============================] - 0s 16ms/step
 скука. скучно словно я с севера что я с севера что я с севера что я с севера что я с севера что я с севе


### Марковская цепь

In [138]:
def data_to_X_Y_markov(data, n):
    ngrams_text = list(ngrams(data, n + 1))
    size_ngrams = len(ngrams_text)

    x = []
    y = []
    for window, index in zip(ngrams_text, range(size_ngrams)):
        x.append(''.join(window[:n]))
        y.append(window[-1])

    return np.array(x), np.array(y)

def read_datat_markov(file_name, n):
    preprocessing(file_name)
    with open('data/' + file_name + '_clear.txt', 'r') as f:
        text = f.read()
        x_line, y_line = data_to_X_Y_markov(text, n)
        return x_line, y_line

def get_markov(x_train, y_train):
    nodes = defaultdict(lambda: defaultdict(lambda: 0))
    for sentence, symbol in zip(x_train, y_train):
        nodes[sentence][symbol] += 1
    return nodes

In [139]:
x_markov, y_markov = read_datat_markov(file_name, n)
nodes = get_markov(x_markov, y_markov)

### Предсказания

In [193]:
def one_hot_to_indexes(a):
    return ''.join(index_to_char[np.argmax(i)] for i in a)

def markov_prediction(nodes, test):
    node = nodes[one_hot_to_indexes(test[0])]
    weights = np.array([w for w in node.values()])

    if len(weights) == 0:
        return index_to_one_hot(char_to_index['.'], len(char_to_index))

    index = node[np.argmax(weights)]
    return index_to_one_hot(char_to_index[[w for w in node][index]], len(char_to_index))

In [194]:
random_generation(file_name, lambda x: markov_prediction(nodes, x), n)

 ведь моя отрада что вовек я любил не один и калитку осеннего сада и опавшие листья с рябин
 ведь
 ведь нам подпою по белу сарафан на подоле. у оврага за плетнями ходит таня ввечер. роса вот уж вечер. ро


In [195]:
random_generation(file_name, lambda x: markov_prediction(nodes, x), n)

 душегубкою змеею развилась ее коса
 душе
 душегубкою змеею развилась речка у милашки малые голодна. воробышки игривые как зимой у печки. и березы 


In [196]:
random_generation(file_name, lambda x: markov_prediction(nodes, x), n)

 но горя ревнивою отвагой будет вслух насвистывать до дома ну и что ж помру себе бродягой на земле и это нам знакомо 
 но г
 но глаза увидевшие землю ни тихих ласковый друг . не нашлось мое колечко щука с ним милашки малые голодн
